Marcin Wardyński  
czwartek, 8:00

### Laboratorium 5: Klasyfikacja

In [1]:
import lab5_utils as utils

corpus, queries, qrels = utils.load_fiqa()

/Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#TODO check the intersection between test, validation and test. Reconsider sizes of the datasets.

In [3]:
from transformers import AutoTokenizer

pl_tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
separator = pl_tokenizer.sep_token

In [4]:

vate_query_to_corpus_dict = utils.prepare_fiqa_qrels(qrels, ['validation', 'test'])
no_vate_corpus_dict = utils.prepare_fiqa_corpus_related_to_selected_subsets(corpus, vate_query_to_corpus_dict, False)

In [5]:
train_index_name = "mw_nlp_lab5_train"
fts_url, train_index_url = utils.create_fts_index(train_index_name)
utils.bulk_load(fts_url, train_index_name, no_vate_corpus_dict)

Index 'mw_nlp_lab5_train' deleted successfully.
Index created.
All documents indexed successfully.


In [6]:
va_query_to_corpus_dict = utils.prepare_fiqa_qrels(qrels, ['validation'])
va_corpus_dict = utils.prepare_fiqa_corpus_related_to_selected_subsets(corpus, va_query_to_corpus_dict, True)

validation_index_name = "mw_nlp_lab5_validation"
fts_url, validation_index_url = utils.create_fts_index(validation_index_name)
utils.bulk_load(fts_url, validation_index_name, va_corpus_dict)

Index 'mw_nlp_lab5_validation' deleted successfully.
Index created.
All documents indexed successfully.


In [7]:
from datasets import Dataset

def collect_training_dataset(index_url, corpus_dict, subsets, separator):
    query_to_corpus_dict = utils.prepare_fiqa_qrels(qrels, subsets)
    queries_dict = utils.prepare_fiqa_queries_for_selected_subset(queries, query_to_corpus_dict)

    results = []

    for q_id, q_text in queries_dict.items():
        collect_passages_for_query(q_id, q_text, corpus_dict, query_to_corpus_dict, index_url, results, separator)

    return Dataset.from_list(results)

def collect_passages_for_query(q_id, q_text, p_dict, q_to_p_dict, index_url, results, separator):
    q_p_ids = list(q_to_p_dict[q_id].keys())

    for q_p_id in q_p_ids:
        results.append(convert_to_json(q_text, p_dict[q_p_id], 1, separator))
        
    fts_results = utils.find_for_phrase_with_exclusion(index_url, q_text, 'text', 3*len(q_p_ids), q_p_ids)
    for fts_result_id in fts_results:
        results.append(convert_to_json(q_text, p_dict[fts_result_id], 0, separator))

def convert_to_json(q_text, p_text, label, separator):
    return {
            "text": f"{q_text}{separator}{p_text}",
            "label": label,
        }
        

In [8]:
train_dataset = collect_training_dataset(train_index_url, no_vate_corpus_dict, ['train'], separator)

In [9]:
validation_dataset = collect_training_dataset(validation_index_url, va_corpus_dict, ['validation'], separator)

In [10]:
from datasets import DatasetDict

datasets = DatasetDict({"train": train_dataset, "validation": validation_dataset})
datasets.save_to_disk("./question-passage-classification")


Saving the dataset (0/1 shards):   0%|          | 0/56664 [00:00<?, ? examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 4952/4952 [00:00<00:00, 182805.63 examples/s]


### Przygotowanie modelu

In [11]:
def tokenize_function(examples):
    return pl_tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = datasets.map(tokenize_function, batched=True)
tokenized_datasets["train"]

Map: 100%|██████████| 4952/4952 [00:01<00:00, 4056.93 examples/s]


Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 56664
})

In [12]:
example = tokenized_datasets["train"][0]
print(example["text"])
print(example["input_ids"])
print(example["attention_mask"])
print("|".join(pl_tokenizer.convert_ids_to_tokens(list(example["input_ids"]))))

print(len([e for e in example["input_ids"] if e != 1]))
print(len([e for e in example["attention_mask"] if e == 1]))

Co jest uważane za wydatek służbowy w podróży służbowej?</s>Wytyczne IRS dotyczące tematu. Ogólnie rzecz biorąc, najlepsze, co mogę powiedzieć, to to, że Twój wydatek biznesowy może podlegać odliczeniu. Ale to zależy od okoliczności i tego, co chcesz odliczyć. Podróże Podatnicy, którzy wyjeżdżają z domu w celach służbowych, mogą odliczyć związane z tym wydatki, w tym koszty dotarcia do miejsca docelowego, koszty zakwaterowania i wyżywienia oraz inne zwykłe i niezbędne wydatki. Podatnicy są uważani za „wyjeżdżających poza dom”, jeśli ich obowiązki wymagają od nich przebywania poza domem znacznie dłużej niż zwykły dzień pracy i muszą spać lub odpoczywać, aby sprostać wymogom pracy. Można odliczyć rzeczywisty koszt posiłków i nieprzewidziane wydatki lub skorzystać ze standardowej diety żywieniowej i obniżonych wymogów ewidencji. Niezależnie od zastosowanej metody odliczenia posiłków są zazwyczaj ograniczone do 50 procent, jak wspomniano wcześniej. Jako koszt można zgłaszać tylko rzeczywis

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "allegro/herbert-base-cased", num_labels=2
)

model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allegro/herbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [14]:
from transformers import TrainingArguments

arguments = TrainingArguments(
    output_dir="./output",
    do_train=True,
    do_eval=True,
    eval_strategy="steps",
    eval_steps=300,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-05,
    num_train_epochs=1,
    logging_first_step=True,
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="epoch",
    fp16=False,
)

In [16]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [17]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42),
    eval_dataset=tokenized_datasets["validation"].shuffle(seed=42),
    compute_metrics=compute_metrics,
)

In [19]:
%load_ext tensorboard
%tensorboard --logdir gdrive/MyDrive/poquad/output/runs

Reusing TensorBoard on port 6006 (pid 48284), started 18:13:54 ago. (Use '!kill 48284' to kill it.)

In [ ]:
trainer.train()

In [18]:
import importlib
import lab5_utils
importlib.reload(lab5_utils)

<module 'lab5_utils' from '/Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/lab5/lab5_utils.py'>